In [1]:
#Load libraries
import pandas as pd
import os
import re
from Bio import SeqIO

In [ ]:
#Reault from the pipeline
mutation_data = pd.read_csv("Mutation_data.csv")
mutation_data

,Contig,Type,SNP_POS,Start,End,Ref,Alt,ORF,Annotation,original codon,original AA,mutated codon,mutated AA,Mutation,FrameShift
0,NODE_1,SNP,5796,5436,6389,A,C,312210_A14_00005,Protease 7,AAT,N,CAT,H,Missense,NaN
1,NODE_1,SNP,6030,5436,6389,T,C,312210_A14_00005,Protease 7,TCT,S,CCT,P,Missense,NaN
2,NODE_1,SNP,6042,5436,6389,C,T,312210_A14_00005,Protease 7,CGT,R,TGT,C,Missense,NaN
3,NODE_1,SNP,6106,5436,6389,C,T,312210_A14_00005,Protease 7,GCC,A,GTC,V,Missense,NaN
4,NODE_1,SNP,6111,5436,6389,T,A,312210_A14_00005,Protease 7,TGT,C,AGT,S,Missense,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,NODE_43,SNP,2277,1402,2430,G,A,312210_A14_04974,hypothetical protein,GCG,A,GCA,A,Silence,NaN
621,NODE_45,SNP,1752,1422,2060,G,T,312210_A14_04981,hypothetical protein,GCT,A,TCT,S,Missense,NaN
622,NODE_3,INSERTION,221760,221363,221803,TGGAGGA,TGGAGGAGGA,312210_A14_01262,Peptidyl-lysine N-acetyltransferase YiaC,NaN,NaN,NaN,NaN,NaN,INSERTION OF 1.0 CODONS
623,NODE_6,DELETION,21708,21626,21841,CGGAT,C,312210_A14_02376,hypothetical protein,NaN,NaN,NaN,NaN,NaN,FRAMESHIFT


In [ ]:
#Get info from mutated gff
mut_gff = pd.read_table("Mutated_genome.gff", comment="#", header=None,
                    names=["contig", "type", "start", "end", "strain", "description"], usecols=[0,2,3,4,6,8])
mut_gff = mut_gff[mut_gff["type"] == "CDS"]

#Usually ORF determination is within the 'locus_tag' delimitation on GFFs (please check your data before running this command)

mut_gff["ORF"] = mut_gff["description"].apply(lambda x: re.findall(r";locus_tag=(.+?);", x)[0])

#The same applies for the annotation, that is usually within the 'product' delimitation

mut_gff["Annotation"] = mut_gff["description"].apply(lambda x: re.findall(r";product=(.+?)$", x)[0])
mut_gff

,contig,type,start,end,strain,description,ORF,Annotation
3,NODE_1,CDS,678,863,-,ID=312211_A14mutant_00004;Name=csrA;db_xref=CO...,312211_A14mutant_00004,Carbon storage regulator
4,NODE_1,CDS,1098,3728,-,ID=312211_A14mutant_00005;eC_number=6.1.1.7;Na...,312211_A14mutant_00005,Alanine--tRNA ligase
5,NODE_1,CDS,3856,4356,-,ID=312211_A14mutant_00006;Name=recX;db_xref=CO...,312211_A14mutant_00006,Regulatory protein RecX
6,NODE_1,CDS,4425,5486,-,ID=312211_A14mutant_00007;Name=recA;db_xref=CO...,312211_A14mutant_00007,Protein RecA
7,NODE_1,CDS,5566,6063,-,ID=312211_A14mutant_00008;eC_number=3.5.1.42;N...,312211_A14mutant_00008,Nicotinamide-nucleotide amidohydrolase PncC
...,...,...,...,...,...,...,...,...
5000,NODE_64,CDS,226,528,-,ID=312211_A14mutant_04999;inference=ab initio ...,312211_A14mutant_04999,IS3 family transposase ISEc52
5002,NODE_67,CDS,45,338,+,ID=312211_A14mutant_05001;inference=ab initio ...,312211_A14mutant_05001,hypothetical protein
5005,NODE_73,CDS,69,422,+,ID=312211_A14mutant_05004;inference=ab initio ...,312211_A14mutant_05004,IS3 family transposase ISEc52
5006,NODE_85,CDS,93,344,+,ID=312211_A14mutant_05005;inference=ab initio ...,312211_A14mutant_05005,hypothetical protein


In [ ]:
equiv_data = pd.read_csv("Equivalent_read_mapping.csv") #From the get_data_from_concat_sam.ipynb

#Carefull here, this adjusts are done because of how I renamed the headers in both fasta files
equiv_data["contig_name_mutated"] = equiv_data["contig_name_mutated"].apply(lambda x: x.replace("Mutated_contig", "NODE"))
equiv_data["contig_name_wild"] = equiv_data["contig_name_wild"].apply(lambda x: x.replace("Wild_contig", "NODE"))
equiv_data

,read_name,contig_name_wild,read_start_wild,read_end_wild,contig_name_mutated,read_start_mutated,read_end_mutated,equivalente_BLAST
0,MNG005:30:AAGWH3HM5:1:1202:50365:47440,NODE_15,68274,68525,NODE_15,45194,45445,True
1,MNG005:30:AAGWH3HM5:1:2114:61934:1757,NODE_21,8119,8370,NODE_18,8119,8370,True
2,MNG005:30:AAGWH3HM5:1:1404:62048:43483,NODE_3,409799,410039,NODE_3,106209,106449,True
3,MNG005:30:AAGWH3HM5:1:1405:14880:21882,NODE_15,29039,29290,NODE_15,84429,84680,True
4,MNG005:30:AAGWH3HM5:1:1113:34686:36800,NODE_4,180705,180956,NODE_4,180704,180955,True
...,...,...,...,...,...,...,...,...
486819,MNG005:30:AAGWH3HM5:1:2102:28968:41798,NODE_40,3246,3497,NODE_20,35556,35807,True
486820,MNG005:30:AAGWH3HM5:1:2213:52239:49390,NODE_3,18922,19173,NODE_3,497075,497326,True
486821,MNG005:30:AAGWH3HM5:1:2301:26828:17622,NODE_1,170796,171047,NODE_2,412661,412912,True
486822,MNG005:30:AAGWH3HM5:1:2411:45271:19553,NODE_1,199159,199410,NODE_2,384298,384549,True


In [5]:
import numpy as np
import re
def normalize_contig_name(name):
    # Remove espaços, padroniza para maiúsculas e remove sufixos indesejados
    return re.sub(r'\s+', '', str(name)).upper()

# Normalizar nomes de contigs em todos os dataframes relevantes
equiv_data["contig_name_wild"] = equiv_data["contig_name_wild"].apply(normalize_contig_name)
equiv_data["contig_name_mutated"] = equiv_data["contig_name_mutated"].apply(normalize_contig_name)
mut_gff["contig"] = mut_gff["contig"].apply(normalize_contig_name)
# Se mutation_data tiver coluna de contig, normalizar também
if "Contig" in mutation_data.columns:
    mutation_data["Contig"] = mutation_data["Contig"].apply(normalize_contig_name)

# Filtrar apenas equivalências válidas: ambos os contigs presentes e não nulos
valid_equiv = equiv_data["contig_name_wild"].notnull() & equiv_data["contig_name_mutated"].notnull()
equiv_data = equiv_data[valid_equiv].copy()

equiv_list = equiv_data[["contig_name_wild","read_start_wild", "read_end_wild","contig_name_mutated", "read_start_mutated", "read_end_mutated"]].values.tolist()

# Converter para arrays numpy para operações vetorizadas
equiv_array = np.array(equiv_list, dtype=object)
wild_contigs_array = equiv_array[:, 0]
wild_starts = equiv_array[:, 1].astype(int)
wild_ends = equiv_array[:, 2].astype(int)
mut_contigs_array = equiv_array[:, 3]
mut_starts = equiv_array[:, 4].astype(int)
mut_ends = equiv_array[:, 5].astype(int)

# Garantir que todos os intervalos estejam no formato correto (start <= end)
wild_starts, wild_ends = np.minimum(wild_starts, wild_ends), np.maximum(wild_starts, wild_ends)
mut_starts, mut_ends = np.minimum(mut_starts, mut_ends), np.maximum(mut_starts, mut_ends)

# Criar arrays para os resultados
mut_annot_results = np.full(len(equiv_list), "Intergenic", dtype=object)
orf_results = np.full(len(equiv_list), "Intergenic", dtype=object)

for index, row in mut_gff.iterrows():
    mut_contig = row["contig"]
    orf_start = min(row["start"], row["end"])
    orf_end = max(row["start"], row["end"])
    annot = row["Annotation"]
    ORF = row["ORF"]

    # Encontrar índices onde o contig MUTATED corresponde
    contig_matches = np.where(mut_contigs_array == mut_contig)[0]

    if len(contig_matches) > 0:
        # Verificar sobreposição para esses índices no genoma mutado
        mask = (mut_starts[contig_matches] <= orf_end) & (mut_ends[contig_matches] >= orf_start)
        matching_indices = contig_matches[mask]
        if len(matching_indices) > 0:
            mut_annot_results[matching_indices] = annot
            orf_results[matching_indices] = ORF

# Atualizar o DataFrame de uma vez
equiv_data["Mut_Annot"] = mut_annot_results
equiv_data["ORF"] = orf_results

# Diagnóstico: quantos equivalentes válidos e anotados
print(f"Total de regiões equivalentes válidas: {len(equiv_data)}")
print(f"Com anotação diferente de Intergenic: {(equiv_data['Mut_Annot'] != 'Intergenic').sum()}")

Total de regiões equivalentes válidas: 486824
Com anotação diferente de Intergenic: 464219


In [7]:
teste = equiv_data.copy()

teste

,read_name,contig_name_wild,read_start_wild,read_end_wild,contig_name_mutated,read_start_mutated,read_end_mutated,equivalente_BLAST,Mut_Annot,ORF
0,MNG005:30:AAGWH3HM5:1:1202:50365:47440,NODE_15,68274,68525,NODE_15,45194,45445,True,Biotin transporter,312211_A14mutant_04197
1,MNG005:30:AAGWH3HM5:1:2114:61934:1757,NODE_21,8119,8370,NODE_18,8119,8370,True,Aromatic-amino-acid aminotransferase,312211_A14mutant_04449
2,MNG005:30:AAGWH3HM5:1:1404:62048:43483,NODE_3,409799,410039,NODE_3,106209,106449,True,Fe(2+) transport protein A,312211_A14mutant_01373
3,MNG005:30:AAGWH3HM5:1:1405:14880:21882,NODE_15,29039,29290,NODE_15,84429,84680,True,Lipid III flippase,312211_A14mutant_04236
4,MNG005:30:AAGWH3HM5:1:1113:34686:36800,NODE_4,180705,180956,NODE_4,180704,180955,True,Intermembrane transport protein YebT,312211_A14mutant_01919
...,...,...,...,...,...,...,...,...,...,...
486819,MNG005:30:AAGWH3HM5:1:2102:28968:41798,NODE_40,3246,3497,NODE_20,35556,35807,True,hypothetical protein,312211_A14mutant_04605
486820,MNG005:30:AAGWH3HM5:1:2213:52239:49390,NODE_3,18922,19173,NODE_3,497075,497326,True,Intergenic,Intergenic
486821,MNG005:30:AAGWH3HM5:1:2301:26828:17622,NODE_1,170796,171047,NODE_2,412661,412912,True,Cytochrome bd-I ubiquinol oxidase subunit 1,312211_A14mutant_01092
486822,MNG005:30:AAGWH3HM5:1:2411:45271:19553,NODE_1,199159,199410,NODE_2,384298,384549,True,Putative isomerase YbhH,312211_A14mutant_01055


In [8]:
equiv_mapped_list = teste[["contig_name_wild", "read_start_wild", "read_end_wild", "read_start_mutated", "read_end_mutated", "Mut_Annot", "contig_name_mutated", "ORF"]].values.tolist()

for index, row in mutation_data.iterrows():
    contig = row["Contig"]
    for contig_name, read_start, read_end, Mread_start, Mread_end, Mut_Annot, Mcontig_name, orf in equiv_mapped_list:
        if contig == contig_name.replace("Wild_contig", "NODE") and (read_start <= row["SNP_POS"] <= read_end):
            mutation_data.loc[index, "Mutated Annotation"] = Mut_Annot
            mutation_data.loc[index, "Mutated Contig"] = Mcontig_name
            mutation_data.loc[index, "Mutated ORF"] = orf
            mutation_data.loc[index, "Mutated_read_Start"] = Mread_start
            mutation_data.loc[index, "Mutated_read_End"] = Mread_end
mutation_data

,Contig,Type,SNP_POS,Start,End,Ref,Alt,ORF,Annotation,original codon,original AA,mutated codon,mutated AA,Mutation,FrameShift,Mutated Annotation,Mutated Contig,Mutated ORF,Mutated_read_Start,Mutated_read_End
0,NODE_1,SNP,5796,5436,6389,A,C,312210_A14_00005,Protease 7,AAT,N,CAT,H,Missense,NaN,Protease 7,NODE_2,312211_A14mutant_01244,577815.0,578066.0
1,NODE_1,SNP,6030,5436,6389,T,C,312210_A14_00005,Protease 7,TCT,S,CCT,P,Missense,NaN,Protease 7,NODE_2,312211_A14mutant_01244,577441.0,577692.0
2,NODE_1,SNP,6042,5436,6389,C,T,312210_A14_00005,Protease 7,CGT,R,TGT,C,Missense,NaN,Protease 7,NODE_2,312211_A14mutant_01244,577441.0,577692.0
3,NODE_1,SNP,6106,5436,6389,C,T,312210_A14_00005,Protease 7,GCC,A,GTC,V,Missense,NaN,Protease 7,NODE_2,312211_A14mutant_01244,577441.0,577692.0
4,NODE_1,SNP,6111,5436,6389,T,A,312210_A14_00005,Protease 7,TGT,C,AGT,S,Missense,NaN,Protease 7,NODE_2,312211_A14mutant_01244,577441.0,577692.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,NODE_43,SNP,2277,1402,2430,G,A,312210_A14_04974,hypothetical protein,GCG,A,GCA,A,Silence,NaN,NaN,NaN,NaN,NaN,NaN
621,NODE_45,SNP,1752,1422,2060,G,T,312210_A14_04981,hypothetical protein,GCT,A,TCT,S,Missense,NaN,hypothetical protein,NODE_46,312211_A14mutant_04981,1599.0,1849.0
622,NODE_3,INSERTION,221760,221363,221803,TGGAGGA,TGGAGGAGGA,312210_A14_01262,Peptidyl-lysine N-acetyltransferase YiaC,NaN,NaN,NaN,NaN,NaN,INSERTION OF 1.0 CODONS,Peptidyl-lysine N-acetyltransferase YiaC,NODE_3,312211_A14mutant_01542,294282.0,294533.0
623,NODE_6,DELETION,21708,21626,21841,CGGAT,C,312210_A14_02376,hypothetical protein,NaN,NaN,NaN,NaN,NaN,FRAMESHIFT,hypothetical protein,NODE_6,312211_A14mutant_02570,26049.0,26304.0


In [9]:
mutation_data.rename(columns={"Contig": "Wild Contig", "SNP_POS": "Wild SNP Position", "Ref":"Wild SNP Nucleotide", "Alt":"Mutated SNP nucleotide",
                              "ORF":"Wild ORF", "Annotation":"Wild Annotation"}, inplace=True)

mutation_data = mutation_data.dropna(subset=["Mutated Contig"])

final_mut_data = mutation_data[["Wild Contig", "Mutated Contig","Wild SNP Position", "Wild SNP Nucleotide", "Mutated SNP nucleotide", "original codon","mutated codon","original AA","mutated AA","Mutation","FrameShift","Wild ORF", "Mutated ORF","Wild Annotation", "Mutated Annotation"]]
final_mut_data["Wild Contig"] = final_mut_data["Wild Contig"].apply(lambda x: x.replace("NODE","Wild_contig"))
final_mut_data["Mutated Contig"] = final_mut_data["Mutated Contig"].apply(lambda x: x.replace("NODE","Mutated_contig"))
final_mut_data

/tmp/ipykernel_418672/3700399559.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_mut_data["Wild Contig"] = final_mut_data["Wild Contig"].apply(lambda x: x.replace("NODE","Wild_contig"))
/tmp/ipykernel_418672/3700399559.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_mut_data["Mutated Contig"] = final_mut_data["Mutated Contig"].apply(lambda x: x.replace("NODE","Mutated_contig"))


,Wild Contig,Mutated Contig,Wild SNP Position,Wild SNP Nucleotide,Mutated SNP nucleotide,original codon,mutated codon,original AA,mutated AA,Mutation,FrameShift,Wild ORF,Mutated ORF,Wild Annotation,Mutated Annotation
0,Wild_contig_1,Mutated_contig_2,5796,A,C,AAT,CAT,N,H,Missense,NaN,312210_A14_00005,312211_A14mutant_01244,Protease 7,Protease 7
1,Wild_contig_1,Mutated_contig_2,6030,T,C,TCT,CCT,S,P,Missense,NaN,312210_A14_00005,312211_A14mutant_01244,Protease 7,Protease 7
2,Wild_contig_1,Mutated_contig_2,6042,C,T,CGT,TGT,R,C,Missense,NaN,312210_A14_00005,312211_A14mutant_01244,Protease 7,Protease 7
3,Wild_contig_1,Mutated_contig_2,6106,C,T,GCC,GTC,A,V,Missense,NaN,312210_A14_00005,312211_A14mutant_01244,Protease 7,Protease 7
4,Wild_contig_1,Mutated_contig_2,6111,T,A,TGT,AGT,C,S,Missense,NaN,312210_A14_00005,312211_A14mutant_01244,Protease 7,Protease 7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,Wild_contig_41,Mutated_contig_40,2517,A,G,GCA,GCG,A,A,Silence,NaN,312210_A14_04965,312211_A14mutant_04964,hypothetical protein,hypothetical protein
593,Wild_contig_41,Mutated_contig_40,2518,T,G,TCC,GCC,S,A,Missense,NaN,312210_A14_04965,312211_A14mutant_04964,hypothetical protein,hypothetical protein
621,Wild_contig_45,Mutated_contig_46,1752,G,T,GCT,TCT,A,S,Missense,NaN,312210_A14_04981,312211_A14mutant_04981,hypothetical protein,hypothetical protein
622,Wild_contig_3,Mutated_contig_3,221760,TGGAGGA,TGGAGGAGGA,NaN,NaN,NaN,NaN,NaN,INSERTION OF 1.0 CODONS,312210_A14_01262,312211_A14mutant_01542,Peptidyl-lysine N-acetyltransferase YiaC,Peptidyl-lysine N-acetyltransferase YiaC


In [10]:
final_mut_data.to_csv("FINAL_Equivalent_read_mapping.csv", index=False)